In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 34.5 MB/s eta 0:00:00


In [ ]:
import os
import random
import shutil
import numpy as np
import moviepy.editor as mp
#import speech_recognition as sr

In [ ]:
import os
import random

# Đường dẫn tới thư mục chứa các file cần xáo trộn và đổi tên
folder_path = '/content/drive/MyDrive/KLTN/data/consensus/video'  # Thay đổi đường dẫn tới thư mục của bạn

# Lấy danh sách tên file trong thư mục
file_names = os.listdir(folder_path)

# Xáo trộn danh sách tên file
random.shuffle(file_names)

# Đổi tên các file theo thứ tự xáo trộn
for i, file_name in enumerate(file_names):
    file_path = os.path.join(folder_path, file_name)
    file_extension = os.path.splitext(file_name)[1]
    new_file_name = f"{i + 1}{file_extension}"  # Đổi tên file thành số thứ tự
    new_file_path = os.path.join(folder_path, new_file_name)
    os.rename(file_path, new_file_path)

# Di chuyển file

In [ ]:
def move_files(source_folder, destination_folder):
    for filename in os.listdir(source_folder):
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)

        shutil.move(source_path, destination_path)
        print("Moved file: {}".format(filename))

In [ ]:
source_folder = '/content/drive/MyDrive/KLTN/data/data_video/test/horrible'
destination_folder = '/content/drive/MyDrive/KLTN/data/data_video/train/horrible'
move_files(source_folder, destination_folder)

# Xóa file

In [ ]:
import os

folder_path = '/content/drive/MyDrive/KLTN/data/data_spec/val/violent'

# Lấy danh sách các file trong thư mục
file_list = os.listdir(folder_path)

# Xóa từng file trong danh sách
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    os.remove(file_path)

print("Đã xóa hết các file trong thư mục.")

Đã xóa hết các file trong thư mục.


# Chia train, test, val

In [ ]:
def split_videos(train_folder, test_folder, val_folder, test_ratio=0.1, val_ratio=0.1):

    video_list = os.listdir(train_folder)
    random.shuffle(video_list)

    total_videos = len(video_list)
    num_test_videos = int(total_videos * test_ratio)
    num_val_videos = int(total_videos * val_ratio)

    for i in range(num_test_videos):
        video_name = video_list[i]
        video_path = os.path.join(train_folder, video_name)
        destination_path = os.path.join(test_folder, video_name)
        shutil.move(video_path, destination_path)
        print("Moved file '{}' to {}".format(video_name, test_folder))

    for i in range(num_test_videos, num_test_videos + num_val_videos):
        video_name = video_list[i]
        video_path = os.path.join(train_folder, video_name)
        destination_path = os.path.join(val_folder, video_name)
        shutil.move(video_path, destination_path)
        print("Moved file '{}' to {}".format(video_name, val_folder))

In [ ]:
train_folder = '/content/drive/MyDrive/KLTN/data/data_video/train/horrible'
test_folder = '/content/drive/MyDrive/KLTN/data/data_video/test/horrible'
val_folder = '/content/drive/MyDrive/KLTN/data/data_video/val/horrible'

test_ratio = 0.1
val_ratio = 0.1
split_videos(train_folder, test_folder, val_folder, test_ratio, val_ratio)

# Đếm file

In [ ]:
import os

folder_path = '/content/drive/MyDrive/KLTN/data/data_video/test/offensive'

# Lấy danh sách các file trong thư mục
file_list = os.listdir(folder_path)

# Đếm số lượng file
num_files = len(file_list)

print("Số lượng file trong thư mục:", num_files)

Số lượng file trong thư mục: 31


# Đổi tên file


In [ ]:
import os

# Đường dẫn đến thư mục chứa video
data_dir = '/content/drive/MyDrive/KLTN/data/consensus/video'

video_files = os.listdir(data_dir)

for i, video_file in enumerate(video_files):
    video_path = os.path.join(data_dir, video_file)
    new_video_name = '{:04d}.mp4'.format(i + 1)  # Định dạng tên mới: 0001.mp4, 0002.mp4, ...
    new_video_path = os.path.join(data_dir, new_video_name)
    os.rename(video_path, new_video_path)

# Speech to Text


In [ ]:
import csv
import speech_recognition as sr
from moviepy.editor import VideoFileClip

In [ ]:
def extract_video_text(input_directory, label, output_csv):
    video_directory = os.path.join(input_directory, label)
    videos = sorted(os.listdir(video_directory))

    with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)

        for video in videos:
            video_path = os.path.join(video_directory, video)

            # Trích xuất văn bản từ âm thanh của video
            recognizer = sr.Recognizer()
            video_clip = VideoFileClip(video_path)
            audio_clip = video_clip.audio
            audio_path = 'temp.wav'
            audio_clip.write_audiofile(audio_path)
            text = ""

            try:
                with sr.AudioFile(audio_path) as audio:
                    audio_data = recognizer.record(audio)
                    text = recognizer.recognize_google(audio_data, language='vi-VN')
            except sr.UnknownValueError:
                print(f"Không thể nhận dạng văn bản từ file {video}. Bỏ qua...")

            os.remove(audio_path)

            writer.writerow([video, label, text])

    print("Trích xuất và lưu văn bản thành công.")

# test

In [ ]:
output_csv = '/content/drive/MyDrive/KLTN/data/data_text/test.csv'
with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name', 'label', 'text'])

In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/test'
label = 'normal'

extract_video_text(input_directory, label, output_csv)

MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


...

MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0030.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Trích xuất và lưu văn bản thành công.


In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/test'
label = 'horrible'

extract_video_text(input_directory, label, output_csv)

MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0001.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


...

Không thể nhận dạng văn bản từ file 0020.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0021.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Trích xuất và lưu văn bản thành công.


In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/test'
label = 'offensive'

extract_video_text(input_directory, label, output_csv)

MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0001.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0002.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


...


Không thể nhận dạng văn bản từ file 0029.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0030.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Trích xuất và lưu văn bản thành công.


In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/test'
label = 'pornographic'

extract_video_text(input_directory, label, output_csv)

MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0001.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.
Không thể nhận dạng văn bản từ file 0002.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.

...

MoviePy - Done.
Không thể nhận dạng văn bản từ file 0029.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.
Không thể nhận dạng văn bản từ file 0030.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.
Không thể nhận dạng văn bản từ file 0031.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Trích xuất và lưu văn bản thành công.


In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/test'
label = 'violent'

extract_video_text(input_directory, label, output_csv)

MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


...

Không thể nhận dạng văn bản từ file 0038.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0040.mp4. Bỏ qua...
Trích xuất và lưu văn bản thành công.


# val

In [ ]:
output_csv = '/content/drive/MyDrive/KLTN/data/data_text/val.csv'
with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name', 'label', 'text'])

In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/val'
label = 'normal'

extract_video_text(input_directory, label, output_csv)

MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


...

MoviePy - Done.


Không thể nhận dạng văn bản từ file 0029.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.
Không thể nhận dạng văn bản từ file 0030.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0031.mp4. Bỏ qua...
Trích xuất và lưu văn bản thành công.


In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/val'
label = 'horrible'

extract_video_text(input_directory, label, output_csv)

MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0001.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.
MoviePy - Writing audio in temp.wav


MoviePy - Done.


...

MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


MoviePy - Done.


Trích xuất và lưu văn bản thành công.


In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/val'
label = 'offensive'

extract_video_text(input_directory, label, output_csv)

MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0002.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


...

MoviePy - Done.


Không thể nhận dạng văn bản từ file 0029.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0031.mp4. Bỏ qua...
Trích xuất và lưu văn bản thành công.


In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/val'
label = 'pornographic'

extract_video_text(input_directory, label, output_csv)

MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0001.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0002.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


...

MoviePy - Done.
Không thể nhận dạng văn bản từ file 0030.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0031.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0032.mp4. Bỏ qua...
Trích xuất và lưu văn bản thành công.


In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/val'
label = 'violent'

extract_video_text(input_directory, label, output_csv)

MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0001.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


MoviePy - Writing audio in temp.wav


...

MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0039.mp4. Bỏ qua...
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Không thể nhận dạng văn bản từ file 0040.mp4. Bỏ qua...
Trích xuất và lưu văn bản thành công.


# train

In [ ]:
output_csv = '/content/drive/MyDrive/KLTN/data/data_text/train.csv'
with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name', 'label', 'text'])

In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/train'
label = 'normal'

extract_video_text(input_directory, label, output_csv)

In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/train'
label = 'horrible'

extract_video_text(input_directory, label, output_csv)

In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/train'
label = 'offensive'

extract_video_text(input_directory, label, output_csv)

In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/train'
label = 'pornographic'

extract_video_text(input_directory, label, output_csv)

In [ ]:
input_directory = '/content/drive/MyDrive/KLTN/data/data_video/train'
label = 'violent'

extract_video_text(input_directory, label, output_csv)

# Spectrogram

In [ ]:
import moviepy.editor as mp
import matplotlib.pyplot as plt
from scipy import signal

def extract_spectrograms_from_videos(video_folder, spectrogram_folder):

    for filename in os.listdir(video_folder):
        if filename.endswith('.mp4'):
            video_path = os.path.join(video_folder, filename)

            video = mp.VideoFileClip(video_path)
            audio = video.audio.to_soundarray()

            # Tạo spectrogram
            frequencies, times, spectrogram = signal.spectrogram(audio[:, 0], fs=video.audio.fps, nperseg=1024, noverlap=512)

            # Vẽ spectrogram
            plt.figure(figsize=(10, 10))
            plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')
            plt.axis('off')
            plt.tight_layout()

            # Lưu spectrogram vào thư mục
            spectrogram_filename = os.path.splitext(filename)[0] + '.png'
            spectrogram_path = os.path.join(spectrogram_folder, spectrogram_filename)
            plt.savefig(spectrogram_path)
            plt.close()

            print("Spectrogram saved for {}".format(filename))

# test

In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/test/horrible'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/test/horrible'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

  return stacker(self.iter_chunks(fps=fps, quantize=quantize,



Spectrogram saved for 0001.mp4
Spectrogram saved for 0002.mp4
Spectrogram saved for 0003.mp4
Spectrogram saved for 0004.mp4
Spectrogram saved for 0005.mp4
Spectrogram saved for 0006.mp4
Spectrogram saved for 0007.mp4
Spectrogram saved for 0008.mp4
Spectrogram saved for 0009.mp4


  plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')



Spectrogram saved for 0010.mp4
Spectrogram saved for 0011.mp4
Spectrogram saved for 0012.mp4
Spectrogram saved for 0013.mp4
Spectrogram saved for 0014.mp4
Spectrogram saved for 0015.mp4
Spectrogram saved for 0016.mp4
Spectrogram saved for 0017.mp4
Spectrogram saved for 0018.mp4
Spectrogram saved for 0019.mp4
Spectrogram saved for 0020.mp4
Spectrogram saved for 0021.mp4
Spectrogram saved for 0022.mp4


In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/test/normal'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/test/normal'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

Spectrogram saved for 0001.mp4
Spectrogram saved for 0002.mp4
Spectrogram saved for 0003.mp4


  plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')



Spectrogram saved for 0004.mp4
Spectrogram saved for 0005.mp4
Spectrogram saved for 0006.mp4
Spectrogram saved for 0007.mp4
Spectrogram saved for 0008.mp4
Spectrogram saved for 0009.mp4
Spectrogram saved for 0010.mp4
Spectrogram saved for 0011.mp4
Spectrogram saved for 0012.mp4
Spectrogram saved for 0013.mp4
Spectrogram saved for 0014.mp4
Spectrogram saved for 0015.mp4
Spectrogram saved for 0016.mp4
Spectrogram saved for 0017.mp4
Spectrogram saved for 0018.mp4
Spectrogram saved for 0019.mp4
Spectrogram saved for 0020.mp4
Spectrogram saved for 0021.mp4
Spectrogram saved for 0022.mp4
Spectrogram saved for 0024.mp4
Spectrogram saved for 0025.mp4
Spectrogram saved for 0026.mp4
Spectrogram saved for 0027.mp4
Spectrogram saved for 0028.mp4
Spectrogram saved for 0029.mp4
Spectrogram saved for 0030.mp4
Spectrogram saved for 0031.mp4
Spectrogram saved for 0023.mp4


In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/test/offensive'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/test/offensive'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

Spectrogram saved for 0001.mp4
Spectrogram saved for 0002.mp4
Spectrogram saved for 0003.mp4
Spectrogram saved for 0004.mp4
Spectrogram saved for 0005.mp4
Spectrogram saved for 0006.mp4
Spectrogram saved for 0007.mp4
Spectrogram saved for 0008.mp4
Spectrogram saved for 0009.mp4
Spectrogram saved for 0010.mp4
Spectrogram saved for 0011.mp4
Spectrogram saved for 0012.mp4
Spectrogram saved for 0014.mp4


  plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')



Spectrogram saved for 0016.mp4
Spectrogram saved for 0017.mp4
Spectrogram saved for 0018.mp4
Spectrogram saved for 0019.mp4
Spectrogram saved for 0020.mp4
Spectrogram saved for 0021.mp4
Spectrogram saved for 0022.mp4
Spectrogram saved for 0023.mp4
Spectrogram saved for 0024.mp4
Spectrogram saved for 0025.mp4
Spectrogram saved for 0027.mp4
Spectrogram saved for 0028.mp4
Spectrogram saved for 0030.mp4
Spectrogram saved for 0031.mp4
Spectrogram saved for 0013.mp4
Spectrogram saved for 0015.mp4
Spectrogram saved for 0026.mp4
Spectrogram saved for 0029.mp4


In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/test/pornographic'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/test/pornographic'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

Spectrogram saved for 0001.mp4
Spectrogram saved for 0002.mp4
Spectrogram saved for 0003.mp4


  plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')



Spectrogram saved for 0004.mp4
Spectrogram saved for 0005.mp4
Spectrogram saved for 0006.mp4
Spectrogram saved for 0008.mp4
Spectrogram saved for 0010.mp4
Spectrogram saved for 0011.mp4
Spectrogram saved for 0012.mp4
Spectrogram saved for 0013.mp4
Spectrogram saved for 0015.mp4
Spectrogram saved for 0016.mp4
Spectrogram saved for 0017.mp4
Spectrogram saved for 0018.mp4
Spectrogram saved for 0019.mp4
Spectrogram saved for 0020.mp4
Spectrogram saved for 0021.mp4
Spectrogram saved for 0022.mp4
Spectrogram saved for 0023.mp4
Spectrogram saved for 0024.mp4
Spectrogram saved for 0025.mp4
Spectrogram saved for 0026.mp4
Spectrogram saved for 0027.mp4
Spectrogram saved for 0028.mp4
Spectrogram saved for 0029.mp4
Spectrogram saved for 0031.mp4
Spectrogram saved for 0032.mp4
Spectrogram saved for 0007.mp4
Spectrogram saved for 0009.mp4
Spectrogram saved for 0014.mp4
Spectrogram saved for 0030.mp4


In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/test/violent'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/test/violent'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

Spectrogram saved for 0001.mp4
Spectrogram saved for 0003.mp4
Spectrogram saved for 0004.mp4
Spectrogram saved for 0005.mp4
Spectrogram saved for 0006.mp4
Spectrogram saved for 0007.mp4
Spectrogram saved for 0008.mp4
Spectrogram saved for 0009.mp4
Spectrogram saved for 0010.mp4
Spectrogram saved for 0011.mp4
Spectrogram saved for 0012.mp4
Spectrogram saved for 0013.mp4
Spectrogram saved for 0015.mp4
Spectrogram saved for 0016.mp4
Spectrogram saved for 0017.mp4
Spectrogram saved for 0018.mp4
Spectrogram saved for 0019.mp4
Spectrogram saved for 0020.mp4
Spectrogram saved for 0021.mp4
Spectrogram saved for 0024.mp4
Spectrogram saved for 0025.mp4
Spectrogram saved for 0026.mp4
Spectrogram saved for 0027.mp4
Spectrogram saved for 0028.mp4


  plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')



Spectrogram saved for 0029.mp4
Spectrogram saved for 0030.mp4
Spectrogram saved for 0031.mp4
Spectrogram saved for 0032.mp4
Spectrogram saved for 0034.mp4
Spectrogram saved for 0036.mp4
Spectrogram saved for 0039.mp4
Spectrogram saved for 0038.mp4
Spectrogram saved for 0022.mp4
Spectrogram saved for 0035.mp4
Spectrogram saved for 0033.mp4
Spectrogram saved for 0023.mp4
Spectrogram saved for 0037.mp4
Spectrogram saved for 0040.mp4


# Val

In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/val/horrible'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/val/horrible'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

Spectrogram saved for 0001.mp4
Spectrogram saved for 0002.mp4
Spectrogram saved for 0003.mp4
Spectrogram saved for 0004.mp4
Spectrogram saved for 0005.mp4


  plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')



Spectrogram saved for 0006.mp4
Spectrogram saved for 0007.mp4
Spectrogram saved for 0008.mp4
Spectrogram saved for 0009.mp4
Spectrogram saved for 0010.mp4
Spectrogram saved for 0011.mp4
Spectrogram saved for 0012.mp4
Spectrogram saved for 0013.mp4
Spectrogram saved for 0014.mp4
Spectrogram saved for 0015.mp4
Spectrogram saved for 0016.mp4
Spectrogram saved for 0017.mp4
Spectrogram saved for 0018.mp4
Spectrogram saved for 0019.mp4
Spectrogram saved for 0020.mp4
Spectrogram saved for 0021.mp4
Spectrogram saved for 0022.mp4


In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/val/normal'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/val/normal'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

Spectrogram saved for 0001.mp4
Spectrogram saved for 0002.mp4
Spectrogram saved for 0003.mp4
Spectrogram saved for 0004.mp4
Spectrogram saved for 0005.mp4
Spectrogram saved for 0006.mp4
Spectrogram saved for 0007.mp4
Spectrogram saved for 0008.mp4
Spectrogram saved for 0009.mp4
Spectrogram saved for 0010.mp4
Spectrogram saved for 0012.mp4
Spectrogram saved for 0013.mp4
Spectrogram saved for 0014.mp4
Spectrogram saved for 0015.mp4
Spectrogram saved for 0016.mp4
Spectrogram saved for 0017.mp4
Spectrogram saved for 0018.mp4
Spectrogram saved for 0019.mp4
Spectrogram saved for 0021.mp4
Spectrogram saved for 0022.mp4
Spectrogram saved for 0023.mp4
Spectrogram saved for 0024.mp4
Spectrogram saved for 0025.mp4
Spectrogram saved for 0026.mp4
Spectrogram saved for 0027.mp4


  plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')



Spectrogram saved for 0028.mp4
Spectrogram saved for 0029.mp4
Spectrogram saved for 0030.mp4
Spectrogram saved for 0031.mp4
Spectrogram saved for 0011.mp4
Spectrogram saved for 0020.mp4


In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/val/offensive'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/val/offensive'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

Spectrogram saved for 0001.mp4
Spectrogram saved for 0002.mp4
Spectrogram saved for 0003.mp4
Spectrogram saved for 0004.mp4
Spectrogram saved for 0005.mp4
Spectrogram saved for 0006.mp4
Spectrogram saved for 0007.mp4
Spectrogram saved for 0008.mp4
Spectrogram saved for 0009.mp4
Spectrogram saved for 0010.mp4
Spectrogram saved for 0011.mp4
Spectrogram saved for 0012.mp4


  plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')



Spectrogram saved for 0013.mp4
Spectrogram saved for 0014.mp4
Spectrogram saved for 0015.mp4
Spectrogram saved for 0016.mp4
Spectrogram saved for 0017.mp4
Spectrogram saved for 0018.mp4
Spectrogram saved for 0019.mp4
Spectrogram saved for 0020.mp4
Spectrogram saved for 0021.mp4
Spectrogram saved for 0022.mp4
Spectrogram saved for 0024.mp4
Spectrogram saved for 0025.mp4
Spectrogram saved for 0026.mp4
Spectrogram saved for 0027.mp4
Spectrogram saved for 0028.mp4
Spectrogram saved for 0029.mp4
Spectrogram saved for 0030.mp4
Spectrogram saved for 0031.mp4
Spectrogram saved for 0023.mp4


In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/val/pornographic'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/val/pornographic'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

  plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')



Spectrogram saved for 0001.mp4
Spectrogram saved for 0002.mp4
Spectrogram saved for 0003.mp4
Spectrogram saved for 0004.mp4
Spectrogram saved for 0005.mp4
Spectrogram saved for 0006.mp4
Spectrogram saved for 0007.mp4
Spectrogram saved for 0008.mp4
Spectrogram saved for 0009.mp4
Spectrogram saved for 0010.mp4
Spectrogram saved for 0011.mp4
Spectrogram saved for 0012.mp4
Spectrogram saved for 0013.mp4
Spectrogram saved for 0014.mp4
Spectrogram saved for 0015.mp4
Spectrogram saved for 0016.mp4
Spectrogram saved for 0017.mp4
Spectrogram saved for 0018.mp4
Spectrogram saved for 0019.mp4
Spectrogram saved for 0020.mp4
Spectrogram saved for 0021.mp4
Spectrogram saved for 0022.mp4
Spectrogram saved for 0023.mp4
Spectrogram saved for 0025.mp4
Spectrogram saved for 0027.mp4
Spectrogram saved for 0028.mp4
Spectrogram saved for 0029.mp4
Spectrogram saved for 0030.mp4
Spectrogram saved for 0031.mp4
Spectrogram saved for 0032.mp4
Spectrogram saved for 0024.mp4
Spectrogram saved for 0026.mp4


In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/val/violent'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/val/violent'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

Spectrogram saved for 0001.mp4
Spectrogram saved for 0002.mp4
Spectrogram saved for 0003.mp4
Spectrogram saved for 0004.mp4
Spectrogram saved for 0005.mp4
Spectrogram saved for 0006.mp4
Spectrogram saved for 0007.mp4
Spectrogram saved for 0008.mp4
Spectrogram saved for 0009.mp4
Spectrogram saved for 0010.mp4
Spectrogram saved for 0011.mp4
Spectrogram saved for 0012.mp4


  plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')



Spectrogram saved for 0013.mp4
Spectrogram saved for 0014.mp4
Spectrogram saved for 0015.mp4
Spectrogram saved for 0016.mp4
Spectrogram saved for 0019.mp4
Spectrogram saved for 0020.mp4
Spectrogram saved for 0021.mp4
Spectrogram saved for 0022.mp4
Spectrogram saved for 0023.mp4
Spectrogram saved for 0024.mp4
Spectrogram saved for 0025.mp4
Spectrogram saved for 0026.mp4
Spectrogram saved for 0027.mp4
Spectrogram saved for 0029.mp4
Spectrogram saved for 0030.mp4
Spectrogram saved for 0031.mp4
Spectrogram saved for 0032.mp4
Spectrogram saved for 0033.mp4
Spectrogram saved for 0034.mp4
Spectrogram saved for 0035.mp4
Spectrogram saved for 0036.mp4
Spectrogram saved for 0037.mp4
Spectrogram saved for 0038.mp4
Spectrogram saved for 0039.mp4
Spectrogram saved for 0040.mp4
Spectrogram saved for 0017.mp4
Spectrogram saved for 0018.mp4
Spectrogram saved for 0028.mp4


# train

In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/train/horrible'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/train/horrible'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/train/normal'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/train/normal'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/train/offensive'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/train/offensive'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/train/pornographic'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/train/pornographic'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)

In [ ]:
video_folder = '/content/drive/MyDrive/KLTN/data/data_video/train/violent'
spectrogram_folder = '/content/drive/MyDrive/KLTN/data/data_spec/train/violent'

extract_spectrograms_from_videos(video_folder, spectrogram_folder)